In [4]:
import pandas as pd

df=pd.read_csv('full_dataset_clean.tsv.gz', sep='\t')
df

,tweet_id,date,time,lang,country_place
0,1212470713338286081,2020-01-01,20:28:39,ru,NaN
1,1212537749485449216,2020-01-02,00:55:01,ru,NaN
2,1212640596508237824,2020-01-02,07:43:42,ru,DE
3,1212707879872016384,2020-01-02,12:11:04,ru,NaN
4,1213076662818091008,2020-01-03,12:36:28,ru,RU
...,...,...,...,...,...
241091714,1360817167415922689,2021-02-14,05:04:30,tl,NaN
241091715,1360817167629885447,2021-02-14,05:04:30,en,NaN
241091716,1360817168821219330,2021-02-14,05:04:30,en,NaN
241091717,1360817170507268096,2021-02-14,05:04:30,en,NaN


In [1]:
from IPython.display import clear_output
!pip install twarc #Twarc
!pip install tweepy # Tweepy 3.8.0
!pip install argparse #Argparse 3.2
!pip install xtract #Xtract 0.1 a3
!pip install wget #Wget 3.2
clear_output()

In [4]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd

dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-01-20/2021-01-20_clean-dataset.tsv.gz?raw=true" #@param {type:"string"}


#Downloads the dataset (compressed in a GZ format)
#!wget dataset_URL -O clean-dataset.tsv.gz
wget.download(dataset_URL, out='clean-dataset.tsv.gz')

#Unzips the dataset and gets the TSV dataset
with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
    with open('clean-dataset.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#Deletes the compressed GZ file
os.unlink("clean-dataset.tsv.gz")

#Gets all possible languages from the dataset
df = pd.read_csv('clean-dataset.tsv',sep="\t")
lang_list = df.lang.unique()
lang_list= sorted(np.append(lang_list,'all'))
lang_picker = widgets.Dropdown(options=lang_list, value="all")
lang_picker

Dropdown(options=('all', 'am', 'ar', 'bg', 'bn', 'bo', 'ca', 'ckb', 'cs', 'cy', 'da', 'de', 'dv', 'el', 'en', …

In [17]:
import json
import tweepy
from tweepy import OAuthHandler

# Authenticate
CONSUMER_KEY = "WLEMGJKOJlo5QQa9feNLFzpl8" #@param {type:"string"}
CONSUMER_SECRET_KEY = "sauCBXzoz0GsV4KMEiod5XsCaTRiJRm3I5TpP3WqwZPClrjWnM" #@param {type:"string"}
ACCESS_TOKEN_KEY = "1653149110407667713-UZk7IXbMT6kIBxBf2eyD6j2zIpNCBg" #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = "6Ga14kNssOKmp2oedIwBXBN1Ytz0laroBbFuOarHIk7e1" #@param {type:"string"}

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)

#The lines below are just to test if the twitter credentials are correct
# Authenticate
#auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

#api = tweepy.API(auth, wait_on_rate_limit=True,
#				   wait_on_rate_limit_notify=True)

#if (not api):
#    print ("Can't Authenticate")
#    sys.exit(-1)

In [6]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = lang_picker.value

#If no language specified, it will get all records from the dataset
if filtered_language == "":
  copyfile('clean-dataset.tsv', 'clean-dataset-filtered.tsv')

#If language specified, it will create another tsv file with the filtered records
else:
  filtered_tw = list()
  current_line = 1
  with open("clean-dataset.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")

    if current_line == 1:
      filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

      for line in tsvreader:
        if line[3] == filtered_language:
          filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
        current_line += 1

  print('\033[1mShowing first 5 tweets from the filtered dataset\033[0m')
  print(filtered_tw[1:(6 if len(filtered_tw) > 6 else len(filtered_tw))])

  with open('clean-dataset-filtered.tsv', 'w') as f_output:
      for item in filtered_tw:
          f_output.write(item)

Showing first 5 tweets from the filtered dataset
['1351757442653294592\t2021-01-20\t05:04:23\ten\tNULL\n', '1351757444033069056\t2021-01-20\t05:04:23\ten\tNULL\n', '1351757446860083202\t2021-01-20\t05:04:24\ten\tNULL\n', '1351757447619375106\t2021-01-20\t05:04:24\ten\tNULL\n', '1351757448219140105\t2021-01-20\t05:04:24\ten\tNULL\n']


In [14]:

from IPython.display import clear_output

!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py -O get_metadata.py

clear_output()

In [19]:
#from tweepy import TweepError

!python get_metadata.py -i clean-dataset-filtered.tsv -o hydrated_tweets -k api_keys.json

#clear_output()

Your twitter api credentials are valid.
hydrated_tweets
tab seperated file, using \t delimiter
total ids: 201374
metadata collection complete
creating master json file
currently getting 0 - 100
Caught the TweepError exception:
 [{'message': 'You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve', 'code': 453}]
Caught the TweepError exception:
 [{'message': 'You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve', 'code': 453}]
Caught the TweepError exception:
 [{'m

In [17]:
import pandas as pd
import glob

path = '/Users/sophiehu/Downloads/folder/*.csv'
all_files = glob.glob(path)

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
merged_df = pd.concat(li, axis=0, ignore_index=True)

merged_df.to_csv('/Users/sophiehu/Downloads/covidtweets.csv', index=False)

In [18]:
tweets=pd.read_csv('covidtweets.csv')
tweets

/Users/sophiehu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,5,16,85) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,gram,counts,covid,191202,162994,coronavirus,183233,224870,297595,288947,...,338424,170023,223328,183891,205017,329716,136428,211307,254575,235019
0,covid 19 cases,7888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,covid 19 pandemic,7312.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,covid 19 vaccine,5349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,positive covid 19,3544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,worse gets better,2854.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624004,NaN,NaN,NaN,NaN,NaN,ministro,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1326.0
624005,NaN,NaN,NaN,NaN,NaN,hari,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1321.0
624006,NaN,NaN,NaN,NaN,NaN,sem,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1320.0
624007,NaN,NaN,NaN,NaN,NaN,worth,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1319.0


In [26]:
import os
import pandas as pd

# Create an empty list to store the dataframes
dfs = []

# Loop through each file in the directory
for filename in os.listdir('/Users/sophiehu/Downloads/folder'):
    # Check if the file ends with '.csv' and contains the string 'April'
    if filename.endswith('.csv') and 'top1000' in filename and 'terms' not in filename:
        # Read the CSV file into a dataframe and append to the list
        df = pd.read_csv('/Users/sophiehu/Downloads/folder/' + filename)
        df['title'] = filename[:-4]  # Add a column for the filename
        dfs.append(df)

# Concatenate the dataframes into a single dataframe
merged_df = pd.concat(dfs, ignore_index=False)

# Write the merged dataframe to a new CSV file
merged_df.to_csv('/Users/sophiehu/Downloads/covid/merged.csv', index=False)

In [27]:
alldf=pd.read_csv('/Users/sophiehu/Downloads/covid/merged.csv')
alldf

,gram,counts,title
0,covid 19 cases,7888,2020-07-21_top1000bigrams
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams
3,positive covid 19,3544,2020-07-21_top1000bigrams
4,worse gets better,2854,2020-07-21_top1000bigrams
...,...,...,...
416001,san raffaele,295,2020-06-01_top1000trigrams
416002,read pass,295,2020-06-01_top1000trigrams
416003,treatment covid,294,2020-06-01_top1000trigrams
416004,casos sospechosos,294,2020-06-01_top1000trigrams


In [28]:
import os
import csv

# Set the folder path
folder_path = "/Users/sophiehu/Downloads/folder"

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and 'top1000terms' in filename:
        csv_file = os.path.join(folder_path, filename)

        # Read the CSV file
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            data = list(reader)

        # Add headers to the data
        data.insert(0, ['gram', 'counts'])

        # Write the data back to the CSV file
        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(data)

In [29]:
import os
import pandas as pd

# Create an empty list to store the dataframes
dfs = []

# Loop through each file in the directory
for filename in os.listdir('/Users/sophiehu/Downloads/folder'):
    # Check if the file ends with '.csv' and contains the string 'April'
    if filename.endswith('.csv') and 'top1000' in filename:
        # Read the CSV file into a dataframe and append to the list
        df = pd.read_csv('/Users/sophiehu/Downloads/folder/' + filename)
        df['title'] = filename[:-4]  # Add a column for the filename
        dfs.append(df)

# Concatenate the dataframes into a single dataframe
merged_df = pd.concat(dfs, ignore_index=False)

# Write the merged dataframe to a new CSV file
merged_df.to_csv('/Users/sophiehu/Downloads/covid/merged.csv', index=False)

In [2]:
import pandas as pd
finaldf=pd.read_csv('/Users/sophiehu/Downloads/covid/merged.csv')
finaldf

,gram,counts,title
0,covid 19 cases,7888,2020-07-21_top1000bigrams
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams
3,positive covid 19,3544,2020-07-21_top1000bigrams
4,worse gets better,2854,2020-07-21_top1000bigrams
...,...,...,...
624209,ministro,1326,2020-05-20_top1000terms
624210,hari,1321,2020-05-20_top1000terms
624211,sem,1320,2020-05-20_top1000terms
624212,worth,1319,2020-05-20_top1000terms


In [3]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
from flair.models import TextClassifier
from flair.data import Sentence

In [5]:
def textblob_sentiment(text):
    return TextBlob(text).sentiment.polarity

In [6]:
def vader_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)

In [7]:
def flair_sentiment(text):
    classifier = TextClassifier.load('en-sentiment')
    sentence = Sentence(text)
    classifier.predict(sentence)
    return sentence.labels[0].score

In [8]:
sentiments = finaldf.copy()
sentiments.dropna(subset=['gram'], inplace=True)

In [9]:
sentiments['textblob_sentiment'] = sentiments['gram'].apply(textblob_sentiment)

In [10]:
sentiments

,gram,counts,title,textblob_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000
...,...,...,...,...
624209,ministro,1326,2020-05-20_top1000terms,0.000000
624210,hari,1321,2020-05-20_top1000terms,0.000000
624211,sem,1320,2020-05-20_top1000terms,0.000000
624212,worth,1319,2020-05-20_top1000terms,0.300000


In [12]:
# split the 'gram' column into 'date' and 'category' columns
sentiments[['date', 'category']] = sentiments['title'].str.split('_', n=1, expand=True)

# format the 'date' column as YYYY-MM-DD
sentiments['date'] = pd.to_datetime(sentiments['date']).dt.date.astype(str)

sentiments

,gram,counts,title,textblob_sentiment,date,category
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams
...,...,...,...,...,...,...
624209,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms
624210,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms
624211,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms
624212,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms


In [76]:
chunkdf

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp..."
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co..."
...,...,...,...,...,...,...,...
624130,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624131,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624132,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624133,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."


In [78]:
currentdf = chunkdf.copy()
currentdf
# for checkpoint loss purposes

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp..."
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co..."
...,...,...,...,...,...,...,...
624130,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624131,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624132,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624133,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."


In [23]:
sentiments['gram'].astype(str)

0            covid 19 cases
1         covid 19 pandemic
2          covid 19 vaccine
3         positive covid 19
4         worse gets better
                ...        
624209             ministro
624210                 hari
624211                  sem
624212                worth
624213                 lack
Name: gram, Length: 624135, dtype: object

In [30]:
import pandas as pd
from tqdm import tqdm
import numpy as np

chunks = np.array_split(sentiments, 1000)

# Iterate over the chunks using tqdm progress bar
for chunk in tqdm(chunks):
    # Apply your code to the current chunk
    chunk['vader_sentiment'] = chunk['gram'].apply(vader_sentiment)

100%|██████████| 1000/1000 [2:42:08<00:00,  9.73s/it]   


In [43]:
chunkdf = pd.concat(chunks, ignore_index=True)
chunkdf.head()

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp..."
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co..."


In [45]:
sentiments['vader_sentiment'] = chunkdf['vader_sentiment']
sentiments

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp..."
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co..."
...,...,...,...,...,...,...,...
624209,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,NaN
624210,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,NaN
624211,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,NaN
624212,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,NaN


In [62]:
fixing = sentiments[sentiments['vader_sentiment'].isna()] # fix NaNs
fixing 
fixing['vader_sentiment'] = fixing['gram'].apply(vader_sentiment)
fixing

<ipython-input-62-560989e89c6d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixing['vader_sentiment'] = fixing['gram'].apply(vader_sentiment)


,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
624135,agree,1416,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."
624136,run,1416,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624137,saúde,1413,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624138,communities,1412,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624139,noticias,1411,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...,...,...,...,...
624209,ministro,1326,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624210,hari,1321,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624211,sem,1320,2020-05-20_top1000terms,0.0,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624212,worth,1319,2020-05-20_top1000terms,0.3,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."


In [83]:
sample = finaldf.sample(n=10)
sample

,gram,counts,title
308114,stop trump,200,2020-10-11_top1000trigrams
463653,day spike,372,2020-06-13_top1000trigrams
60487,since covid 19,274,2020-04-21_top1000trigrams
368437,120 000 people,285,2020-06-21_top1000bigrams
56809,charles highlights coronavirus,114,2020-09-27_top1000bigrams
466368,critically ill,765,2020-09-02_top1000trigrams
479459,confirmados covid 19,320,2020-05-19_top1000trigrams
187763,we,1544,2020-06-07_top1000terms
233534,patient researchers say,232,2020-08-28_top1000bigrams
521458,contact,3919,2020-06-05_top1000terms


In [90]:
for i in tqdm(sample):
    # Apply your code to the current chunk
    sample['flair_sentiment'] = sample['gram'].apply(flair_sentiment)

 40%|████      | 4/10 [01:53<02:49, 28.31s/it]


In [87]:
# based on the above progress bar - it would take approx 14 million seconds - 172 days to process the entire dataset.
# but when look at actual df it's populated? 
# unclear if works for full df - maybe skip

In [91]:
sample

,gram,counts,title,flair_sentiment
308114,stop trump,200,2020-10-11_top1000trigrams,0.975517
463653,day spike,372,2020-06-13_top1000trigrams,0.838872
60487,since covid 19,274,2020-04-21_top1000trigrams,0.995612
368437,120 000 people,285,2020-06-21_top1000bigrams,0.985861
56809,charles highlights coronavirus,114,2020-09-27_top1000bigrams,0.993125
466368,critically ill,765,2020-09-02_top1000trigrams,0.999971
479459,confirmados covid 19,320,2020-05-19_top1000trigrams,0.999747
187763,we,1544,2020-06-07_top1000terms,0.981540
233534,patient researchers say,232,2020-08-28_top1000bigrams,0.942363
521458,contact,3919,2020-06-05_top1000terms,0.987091


In [92]:
# Iterate over the chunks using tqdm progress bar

chunks2 = np.array_split(sentiments, 1000)

for chunk2 in tqdm(chunks2):
    # Apply your code to the current chunk
    chunk2['flair_sentiment'] = chunk2['gram'].apply(flair_sentiment)

  0%|          | 1/1000 [1:31:49<1528:58:11, 5509.80s/it]


KeyboardInterrupt: 

In [ ]:
currentdf['flair_sentiment'] = chunk2df['flair_sentiment']
currentdf

In [96]:
#5279.38s/it so flair would take forever :(

In [94]:
currentdf

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp..."
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co..."
...,...,...,...,...,...,...,...
624130,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624131,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624132,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
624133,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."


In [98]:
currentdf['vader_sentiment']

0         {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1         {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2         {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
3         {'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp...
4         {'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co...
                                ...                        
624130    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
624131    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
624132    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
624133    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
624134    {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...
Name: vader_sentiment, Length: 624135, dtype: object

In [103]:
vaderdf = currentdf.copy()

for i in tqdm(vaderdf):
    # Apply your code to the current chunk
    vaderdf[['neg', 'neu', 'pos', 'compound']] = vaderdf['vader_sentiment'].apply(pd.Series)

  0%|          | 0/624135 [03:08<?, ?it/s]


KeyboardInterrupt: 

In [105]:
sample = currentdf.sample(n=10)
sample

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment
510419,covid19 amp,460,2020-08-19_top1000trigrams,0.0,2020-08-19,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
194395,nothing,2157,2020-09-28_top1000terms,0.0,2020-09-28,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
578036,homenaje víctimas,330,2020-07-17_top1000trigrams,0.0,2020-07-17,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
11050,could,6989,2020-08-08_top1000terms,0.0,2020-08-08,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
106356,redefine relationship,546,2020-09-24_top1000trigrams,0.0,2020-09-24,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
283381,health minister,406,2020-08-21_top1000trigrams,0.0,2020-08-21,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
421821,anti,1432,2020-06-21_top1000terms,0.0,2020-06-21,top1000terms,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
404977,coronavirus surge,268,2020-07-04_top1000trigrams,0.0,2020-07-04,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
29728,dangerous experts rip,198,2020-04-23_top1000trigrams,-0.6,2020-04-23,top1000trigrams,"{'neg': 0.608, 'neu': 0.392, 'pos': 0.0, 'comp..."
468378,24,4296,2020-04-05_top1000terms,0.0,2020-04-05,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [109]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_components(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment

for i in tqdm(sample):
    # Apply your code to the current chunk
    vaderdf[['neg', 'neu', 'pos', 'compound']] = sample['vader_sentiment'].apply(get_sentiment_components).apply(pd.Series)
    
vaderdf

 70%|███████   | 7/10 [00:00<00:00, 18.19it/s]


,gram,counts,title,textblob_sentiment,date,category,vader_sentiment,neg,neu,pos,compound
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp...",NaN,NaN,NaN,NaN
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
624130,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
624131,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
624132,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN,NaN,NaN,NaN
624133,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",NaN,NaN,NaN,NaN


In [110]:
sentiments_df = pd.DataFrame(sample['vader_sentiment'].tolist(), index=sample.index)
sentiments_df.columns = ['sentiment_' + col for col in sentiments_df.columns]
sentiments = pd.concat([sample, sentiments_df], axis=1)
sentiments

,gram,counts,title,textblob_sentiment,date,category,vader_sentiment,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
510419,covid19 amp,460,2020-08-19_top1000trigrams,0.0,2020-08-19,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
194395,nothing,2157,2020-09-28_top1000terms,0.0,2020-09-28,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
578036,homenaje víctimas,330,2020-07-17_top1000trigrams,0.0,2020-07-17,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
11050,could,6989,2020-08-08_top1000terms,0.0,2020-08-08,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
106356,redefine relationship,546,2020-09-24_top1000trigrams,0.0,2020-09-24,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
283381,health minister,406,2020-08-21_top1000trigrams,0.0,2020-08-21,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
421821,anti,1432,2020-06-21_top1000terms,0.0,2020-06-21,top1000terms,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...",1.000,0.000,0.0,-0.3182
404977,coronavirus surge,268,2020-07-04_top1000trigrams,0.0,2020-07-04,top1000trigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000
29728,dangerous experts rip,198,2020-04-23_top1000trigrams,-0.6,2020-04-23,top1000trigrams,"{'neg': 0.608, 'neu': 0.392, 'pos': 0.0, 'comp...",0.608,0.392,0.0,-0.4767
468378,24,4296,2020-04-05_top1000terms,0.0,2020-04-05,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.0000


In [112]:
vaderdf = currentdf.copy()

for i in tqdm(vaderdf):
    # Apply your code to the current chunk
    sentiments_df = pd.DataFrame(vaderdf['vader_sentiment'].tolist(), index=vaderdf.index)
    sentiments_df.columns = ['sentiment_' + col for col in sentiments_df.columns]
sentiments = pd.concat([vaderdf, sentiments_df], axis=1)
sentiments

  0%|          | 7/624135 [00:04<100:37:04,  1.72it/s]


,gram,counts,title,textblob_sentiment,date,category,vader_sentiment,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
0,covid 19 cases,7888,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
1,covid 19 pandemic,7312,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
2,covid 19 vaccine,5349,2020-07-21_top1000bigrams,0.000000,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
3,positive covid 19,3544,2020-07-21_top1000bigrams,0.227273,2020-07-21,top1000bigrams,"{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'comp...",0.000,0.357,0.643,0.5574
4,worse gets better,2854,2020-07-21_top1000bigrams,0.050000,2020-07-21,top1000bigrams,"{'neg': 0.443, 'neu': 0.143, 'pos': 0.414, 'co...",0.443,0.143,0.414,-0.0516
...,...,...,...,...,...,...,...,...,...,...,...
624130,ministro,1326,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
624131,hari,1321,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
624132,sem,1320,2020-05-20_top1000terms,0.000000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
624133,worth,1319,2020-05-20_top1000terms,0.300000,2020-05-20,top1000terms,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.000,0.000,1.000,0.2263


In [113]:
# do one more sentiment analysis type and then group everything by timing to see when is polarizing
# check each type to see if averaging or something else makes the most sense for function

In [114]:
# BERT

In [117]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [122]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [119]:
pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 3.4 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [120]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 843.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 9.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 36.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.2 MB/s eta 0:00:0000:0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.2 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.9-py3-none-any.whl size=1477708 sha256=b41b82e0127142794dfd7e90321d1e7201f2c82c3846270ea3152b8d08672c37
  Stored in directory: /Users/sophiehu/Library/Caches/pip/wheels/59/43/61/f07ddc82e6b3e002bc38d095ca673476fcd7672538d88d88fe
Successfully built jax
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.2
    Uninstalling protobuf-3.20.2:
      Successfully uninstalled protobuf-3.20.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0
Note: you may need to restart the kernel to use updated packages.


In [124]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures


ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [125]:
# had to restart to use package, wouldve lost everything in kernel - started a new notebook

In [126]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [134]:
sentiment_pipeline(sample['gram'].values.tolist())

[{'label': 'NEGATIVE', 'score': 0.5695649981498718},
 {'label': 'NEGATIVE', 'score': 0.9992395639419556},
 {'label': 'NEGATIVE', 'score': 0.9979739785194397},
 {'label': 'POSITIVE', 'score': 0.8508513569831848},
 {'label': 'POSITIVE', 'score': 0.9841240048408508},
 {'label': 'POSITIVE', 'score': 0.9748139977455139},
 {'label': 'NEGATIVE', 'score': 0.9990823268890381},
 {'label': 'POSITIVE', 'score': 0.7778550386428833},
 {'label': 'NEGATIVE', 'score': 0.9885321855545044},
 {'label': 'POSITIVE', 'score': 0.9911569356918335}]